In [1]:
import torch.multiprocessing
import torch.optim as optim
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms

from corrosion_dataset import CorrosionSegmentationDataset
from unet import UNet

torch.multiprocessing.set_start_method("spawn", force=True)  # ✅ Must be the first line

# ✅ Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:
# ✅ Reduce num_workers to prevent crashing
BATCH_SIZE = 16
NUM_WORKERS = 2  # ✅ Reduce for stability (or set to 0 if still crashing)
GRAD_ACCUM_STEPS = 2  # ✅ Gradient accumulation

# Set dataset paths
image_dir = "processed_dataset"
mask_dir = "processed_dataset_masks"

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5]),  # Normalize grayscale images
])

# ✅ Load dataset inside `if __name__ == "__main__"`
dataset = CorrosionSegmentationDataset(image_dir, mask_dir, transform=transform)
train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)

# ✅ Initialize model inside `if __name__ == "__main__"`
model = UNet(num_classes=2).to(device)

D:\001_MLProjects\SEMClassify\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\001_MLProjects\SEMClassify\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FCN_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=FCN_ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
# ✅ Define loss function & optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=1e-4)
scaler = torch.cuda.amp.GradScaler()  # ✅ Mixed Precision (FP16)

# ✅ Training loop
for epoch in range(20):
    model.train()
    total_loss = 0
    correct_pixels = 0
    total_pixels = 0
    optimizer.zero_grad()

    for i, (images, masks) in enumerate(train_loader):
        images, masks = images.to(device), masks.to(device)

        with torch.cuda.amp.autocast():  # ✅ Enable FP16
            outputs = model(images)
            loss = criterion(outputs, masks.long()) / GRAD_ACCUM_STEPS  # ✅ Gradient accumulation

        scaler.scale(loss).backward()

        if (i + 1) % GRAD_ACCUM_STEPS == 0:  # ✅ Update weights after accumulation steps
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

        total_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        correct_pixels += (preds == masks).sum().item()
        total_pixels += masks.numel()

    pixel_accuracy = 100 * correct_pixels / total_pixels
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}, Pixel Accuracy: {pixel_accuracy:.2f}%")

torch.save(model.state_dict(), "unet_corrosion.pth")
print("✅ Training complete. Model saved.")

C:\Users\Kyojuro\AppData\Local\Temp\ipykernel_20288\2934163907.py:4: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()  # ✅ Mixed Precision (FP16)
C:\Users\Kyojuro\AppData\Local\Temp\ipykernel_20288\2934163907.py:17: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():  # ✅ Enable FP16


Epoch 1, Loss: 2.3251, Pixel Accuracy: 64.89%
Epoch 2, Loss: 1.6182, Pixel Accuracy: 83.74%
Epoch 3, Loss: 1.4581, Pixel Accuracy: 86.55%
Epoch 4, Loss: 1.2099, Pixel Accuracy: 88.19%
Epoch 5, Loss: 1.1728, Pixel Accuracy: 88.30%
Epoch 6, Loss: 0.9936, Pixel Accuracy: 90.67%
Epoch 7, Loss: 0.9867, Pixel Accuracy: 90.87%
Epoch 8, Loss: 0.9585, Pixel Accuracy: 91.00%
Epoch 9, Loss: 0.9803, Pixel Accuracy: 91.14%
Epoch 10, Loss: 0.9253, Pixel Accuracy: 91.64%
Epoch 11, Loss: 0.9106, Pixel Accuracy: 91.52%
Epoch 12, Loss: 0.8131, Pixel Accuracy: 92.27%
Epoch 13, Loss: 0.8037, Pixel Accuracy: 92.46%
Epoch 14, Loss: 0.8165, Pixel Accuracy: 92.37%
Epoch 15, Loss: 0.8195, Pixel Accuracy: 92.58%
Epoch 16, Loss: 0.7579, Pixel Accuracy: 92.98%
Epoch 17, Loss: 0.7360, Pixel Accuracy: 93.19%
Epoch 18, Loss: 0.7275, Pixel Accuracy: 93.24%
Epoch 19, Loss: 0.7521, Pixel Accuracy: 93.18%
Epoch 20, Loss: 0.6921, Pixel Accuracy: 93.63%
✅ Training complete. Model saved.
